In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
%cd /content/drive/My Drive/ML Project

/content/drive/My Drive/ML Project


Setup Git

In [ ]:
!git version
!git help

In [8]:
!ls

logfiles.log  logfiles.log.zip	ML_codebase.ipynb  test_generator.ipynb


In [10]:
!echo "# auto-scaling-prediction" >> README.md

In [13]:
!ls

logfiles.log	  ML_codebase.ipynb  test_generator.ipynb
logfiles.log.zip  README.md


In [12]:
!echo "logfiles.log" >> .gitignore

In [14]:
!git init
!git add README.md
!git commit -m "first commit"
!git branch -M main
!git remote add origin https://github.com/Gabin-ishimwe/auto-scaling-prediction.git
!git push -u origin main

hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/drive/MyDrive/ML Project/.git/
Author identity unknown

*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'root@6e70094da281.(none)')
error: src refspec main does not match any
error: failed to push some refs to 'https://github.com/Gabin-

In [ ]:
!git